# Decomposition of Trends

# What dependencies to install


In [ ]:
import pandas as pd
from scipy import interpolate
import numpy as np
import seaborn as sns
import pandas as pd
import numpy as np
from random import gauss
from pandas.plotting import autocorrelation_plot
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.ar_model import AR
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

# Loading data


In [ ]:
abundance_table = pd.read_excel("abundance_table.xlsx")
as_outliers_removed = pd.read_excel("as_outliers_removed_june16.xlsx")

# Creating Dataframes


In [ ]:
#data- Shannon Index AS system data
abundance_table = pd.read_excel("abundance_table.xlsx")
#remove first 4 characters in every column name
abundance_table.columns = abundance_table.columns.str[4:] 
#split string in column to get date and process
abundance_table[['Process','Date']] = abundance_table['le'].str.split('_',expand=True)
abundance_table = abundance_table.drop(columns=['le'])
#keep only AS-1 and AS-2 data
processes = ['AS-1', 'AS-2']
as_abundance_table = abundance_table[abundance_table['Process'].isin(processes)]
as_abundance_table = as_abundance_table.drop(columns=['Process'])
#group by date and find the mean of all values
as_abundance_table = as_abundance_table.groupby(['Date']).mean()
as_abundance_table['sum'] = as_abundance_table.sum(axis=1)
as_abundance_table = as_abundance_table.reset_index()
as_abundance = as_abundance_table.set_index('Date')
#find percentage each column by dividing by sum
as_abundance = as_abundance.div(as_abundance['sum'], axis=0)
as_abundance = as_abundance.drop(columns=['sum'])
#pivot dataframe from wide to long format
as_abundance = as_abundance.reset_index()
as_abundance = as_abundance.melt(id_vars=['Date'], var_name='OTU', value_name='Abundance')
#remove all rows with an abundance less than 0.01
#as_abundance_low_filter = as_abundance[as_abundance['Abundance'] >= 0.0005] 
as_abundance = as_abundance[as_abundance['Abundance'] >= 0.01] 

def shannon_index(x):
    return -1 * np.sum(x*np.log(x))

Shannon_Diversity = as_abundance.groupby('Date')['Abundance'].agg(shannon_index).reset_index()

as_abundance = as_abundance.merge(Shannon_Diversity, on='Date')
as_abundance = as_abundance.rename(columns={'Abundance_x':'Abundance', 'Abundance_y':'Shannon Index'})

as_abundance_table_moved = as_abundance_table.melt(id_vars=['Date'], var_name='OTU', value_name='Counts')

#merging with as_abundance
unfiltered_data = as_abundance.merge(as_abundance_table_moved, on=['Date','OTU'])

unfiltered_data['Date'] = pd.to_datetime(unfiltered_data['Date'])
unfiltered_data = unfiltered_data.set_index('Date')
unfiltered_data = unfiltered_data.sort_values(by='Date')

## Seasonal Decomposition


In [ ]:
unfiltered_data = unfiltered_data
decomposition = sm.tsa.seasonal_decompose(unfiltered_data['Shannon Index'], period = 4) 
figure = decomposition.plot()
plt.show()

## STL Decomposition


In [ ]:
from statsmodels.tsa.seasonal import STL
stl = STL(unfiltered_data['Shannon Index'])
res = stl.fit()
fig = res.plot()

## Shannon Index over time


In [ ]:
#plot Shannon Index over time with angle = 90 on x-axis labels
plt.xticks(rotation=90, size = 5)
sns.lineplot(x='Date', y='Shannon Index', data=unfiltered_data)
plt.title('Shannon Index Over Time')
#add x-axis label
plt.xlabel('Date')
plt.xticks(size = 10)

In [ ]:
#top 10 most abundant species
def get_top_n(group, n=10):
    return group.nlargest(n, columns='Abundance')

# Apply the function to each group
top_n_per_group = unfiltered_data.groupby('Date').apply(get_top_n).reset_index()
top_n_per_group = top_n_per_group.reset_index()
# Extract the values into a list for each group
result = top_n_per_group.groupby('Date')['OTU'].apply(list).to_dict()

#OTU 1 is very common/highly abundant
#filter unfiltered_data by OTU 1
f, ax = plt.subplots(1, 1)
otu_1 = unfiltered_data[unfiltered_data['OTU'] == '1']
sns.lineplot(x='Date', y='Counts', data=otu_1, label="1",)
otu_4 = unfiltered_data[unfiltered_data['OTU'] == '4']
sns.lineplot(x='Date', y='Counts', data=otu_4, label="4",)
otu_2 = unfiltered_data[unfiltered_data['OTU'] == '2']
sns.lineplot(x='Date', y='Counts', data=otu_2, label="2",)
otu_6 = unfiltered_data[unfiltered_data['OTU'] == '6']
sns.lineplot(x='Date', y='Counts', data=otu_6, label="6",)
#add a legend for each lineplot
ax.legend()
#add title
plt.title('Abundance Counts of Top Growing OTUs Over Time')
#add x-axis label
plt.xlabel('Date')
plt.xticks(size = 10)